# Calorie Expenditure Prediction Project

This notebook provides a comprehensive analysis for predicting calorie expenditure using multiple machine learning models.

## Project Overview
- **Goal**: Predict calorie expenditure based on various features
- **Models**: Random Forest, XGBoost, and Ridge Regression
- **Approach**: Complete data analysis, cleaning, feature engineering, and model comparison

## Table of Contents
1. Data Loading and Exploration
2. Data Cleaning and Preprocessing
3. Exploratory Data Analysis
4. Feature Engineering
5. Model Training and Evaluation
6. Model Comparison and Selection

In [ ]:
# Import all necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Machine Learning libraries
from sklearn.model_selection import train_test_split, cross_val_score, learning_curve
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV

# XGBoost
try:
    import xgboost as xgb
    XGBOOST_AVAILABLE = True
except ImportError:
    print("XGBoost not installed. Installing...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "xgboost"])
    import xgboost as xgb
    XGBOOST_AVAILABLE = True

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("All libraries imported successfully!")

## 1. Data Loading and Initial Exploration

In [ ]:
# Load the datasets
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

print("Dataset Shapes:")
print(f"Training data: {train_data.shape}")
print(f"Test data: {test_data.shape}")

# Display basic information about the datasets
print("\n" + "="*50)
print("TRAINING DATA OVERVIEW")
print("="*50)
print("\nFirst 5 rows:")
display(train_data.head())

print("\nData types and missing values:")
train_info = pd.DataFrame({
    'Data Type': train_data.dtypes,
    'Missing Values': train_data.isnull().sum(),
    'Missing %': (train_data.isnull().sum() / len(train_data)) * 100
})
display(train_info)

print("\n" + "="*50)
print("TEST DATA OVERVIEW")
print("="*50)
print("\nFirst 5 rows:")
display(test_data.head())

print("\nData types and missing values:")
test_info = pd.DataFrame({
    'Data Type': test_data.dtypes,
    'Missing Values': test_data.isnull().sum(),
    'Missing %': (test_data.isnull().sum() / len(test_data)) * 100
})
display(test_info)

In [ ]:
# Detailed statistical analysis of the training data
print("STATISTICAL SUMMARY OF TRAINING DATA")
print("="*40)

# Numerical features summary
numerical_cols = train_data.select_dtypes(include=[np.number]).columns
print(f"\nNumerical columns: {list(numerical_cols)}")
display(train_data[numerical_cols].describe())

# Categorical features summary
categorical_cols = train_data.select_dtypes(include=['object']).columns
print(f"\nCategorical columns: {list(categorical_cols)}")

for col in categorical_cols:
    print(f"\n{col}:")
    print(f"  Unique values: {train_data[col].nunique()}")
    print(f"  Values: {train_data[col].unique()[:10]}")
    if train_data[col].nunique() <= 10:
        print(f"  Value counts:")
        display(train_data[col].value_counts())

## 2. Data Cleaning and Preprocessing

In [ ]:
# Check for and handle missing values
print("MISSING VALUES ANALYSIS")
print("="*30)

# Check missing values in both datasets
missing_train = train_data.isnull().sum()
missing_test = test_data.isnull().sum()

print("Missing values in training data:")
print(missing_train[missing_train > 0])

print("\nMissing values in test data:")
print(missing_test[missing_test > 0])

if missing_train.sum() == 0 and missing_test.sum() == 0:
    print("\n✅ No missing values found in either dataset!")
else:
    print("\n⚠️ Missing values detected. Handling them...")
    # Handle missing values if any exist
    # (Add specific handling code here if needed)

# Check for duplicates
print(f"\nDuplicate rows in training data: {train_data.duplicated().sum()}")
print(f"Duplicate rows in test data: {test_data.duplicated().sum()}")

# Remove duplicates if any
if train_data.duplicated().sum() > 0:
    train_data = train_data.drop_duplicates()
    print("Removed duplicate rows from training data")

if test_data.duplicated().sum() > 0:
    test_data = test_data.drop_duplicates()
    print("Removed duplicate rows from test data")

In [ ]:
# Data type optimization and categorical encoding preparation
print("DATA TYPE OPTIMIZATION")
print("="*25)

# Create copies for processing
train_processed = train_data.copy()
test_processed = test_data.copy()

# Store original categorical columns before encoding
original_categorical = train_processed.select_dtypes(include=['object']).columns.tolist()
print(f"Categorical columns to encode: {original_categorical}")

# Initialize encoders dictionary to store for consistency
encoders = {}

# Encode categorical variables consistently across train and test
for col in original_categorical:
    le = LabelEncoder()
    
    # Fit on combined unique values from both train and test
    combined_values = pd.concat([train_processed[col], test_processed[col]]).unique()
    le.fit(combined_values)
    
    # Transform both datasets
    train_processed[col] = le.transform(train_processed[col])
    test_processed[col] = le.transform(test_processed[col])
    
    # Store encoder
    encoders[col] = le
    
    print(f"Encoded {col}: {len(combined_values)} unique values")

print("\n✅ Categorical encoding completed!")
print(f"Training data shape after encoding: {train_processed.shape}")
print(f"Test data shape after encoding: {test_processed.shape}")

## 3. Exploratory Data Analysis (EDA)

In [ ]:
# Target variable analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Distribution of target variable
axes[0,0].hist(train_processed['Calories'], bins=50, alpha=0.7, color='skyblue', edgecolor='black')
axes[0,0].set_title('Distribution of Calories (Target Variable)')
axes[0,0].set_xlabel('Calories')
axes[0,0].set_ylabel('Frequency')

# Box plot for outlier detection
axes[0,1].boxplot(train_processed['Calories'])
axes[0,1].set_title('Box Plot of Calories')
axes[0,1].set_ylabel('Calories')

# Q-Q plot for normality check
from scipy import stats
stats.probplot(train_processed['Calories'], dist="norm", plot=axes[1,0])
axes[1,0].set_title('Q-Q Plot of Calories')

# Log transformation (if needed)
log_calories = np.log1p(train_processed['Calories'])
axes[1,1].hist(log_calories, bins=50, alpha=0.7, color='lightcoral', edgecolor='black')
axes[1,1].set_title('Log-transformed Calories Distribution')
axes[1,1].set_xlabel('Log(Calories + 1)')
axes[1,1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

# Statistical summary of target variable
print("TARGET VARIABLE STATISTICS")
print("="*30)
print(f"Mean: {train_processed['Calories'].mean():.2f}")
print(f"Median: {train_processed['Calories'].median():.2f}")
print(f"Standard Deviation: {train_processed['Calories'].std():.2f}")
print(f"Skewness: {train_processed['Calories'].skew():.3f}")
print(f"Kurtosis: {train_processed['Calories'].kurtosis():.3f}")
print(f"Range: {train_processed['Calories'].min():.2f} - {train_processed['Calories'].max():.2f}")

In [ ]:
# Feature correlation analysis
print("CORRELATION ANALYSIS")
print("="*20)

# Calculate correlation matrix
correlation_matrix = train_processed.drop('id', axis=1).corr()

# Create correlation heatmap
plt.figure(figsize=(14, 10))
sns.heatmap(correlation_matrix, annot=True, fmt='.3f', cmap='RdYlBu_r', 
            center=0, square=True, linewidths=0.5)
plt.title('Feature Correlation Matrix', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# Find features most correlated with target
target_correlations = correlation_matrix['Calories'].abs().sort_values(ascending=False)
print("\nFeatures most correlated with Calories:")
print(target_correlations[1:])  # Exclude self-correlation

# Identify highly correlated feature pairs (potential multicollinearity)
print("\nHighly correlated feature pairs (|correlation| > 0.7):")
high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            high_corr_pairs.append((
                correlation_matrix.columns[i], 
                correlation_matrix.columns[j], 
                correlation_matrix.iloc[i, j]
            ))

for pair in high_corr_pairs:
    print(f"{pair[0]} - {pair[1]}: {pair[2]:.3f}")

if not high_corr_pairs:
    print("No highly correlated feature pairs found.")

In [ ]:
# Feature distribution analysis
features_to_plot = [col for col in train_processed.columns if col not in ['id', 'Calories']]
n_features = len(features_to_plot)
n_cols = 3
n_rows = (n_features + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
axes = axes.flatten() if n_rows > 1 else [axes] if n_rows == 1 else axes

for i, feature in enumerate(features_to_plot):
    if i < len(axes):
        # Check if feature was originally categorical
        if any(feature in original_cat for original_cat in [original_categorical]):
            # Bar plot for categorical features
            value_counts = train_processed[feature].value_counts().head(10)
            axes[i].bar(range(len(value_counts)), value_counts.values, alpha=0.7)
            axes[i].set_title(f'{feature} Distribution (Categorical)')
            axes[i].set_xlabel('Encoded Values')
            axes[i].set_ylabel('Count')
        else:
            # Histogram for numerical features
            axes[i].hist(train_processed[feature], bins=30, alpha=0.7, edgecolor='black')
            axes[i].set_title(f'{feature} Distribution')
            axes[i].set_xlabel(feature)
            axes[i].set_ylabel('Frequency')
        
        axes[i].grid(True, alpha=0.3)

# Hide empty subplots
for i in range(len(features_to_plot), len(axes)):
    axes[i].set_visible(False)

plt.tight_layout()
plt.show()

## 4. Feature Engineering

In [ ]:
# Feature engineering and selection
print("FEATURE ENGINEERING")
print("="*19)

# Create copies for feature engineering
train_features = train_processed.copy()
test_features = test_processed.copy()

# Remove ID column and separate target
X = train_features.drop(['id', 'Calories'], axis=1)
y = train_features['Calories']
X_test = test_features.drop('id', axis=1)

print(f"Original features: {X.columns.tolist()}")
print(f"Number of features: {X.shape[1]}")
print(f"Training samples: {X.shape[0]}")
print(f"Test samples: {X_test.shape[0]}")

# Feature scaling for linear models
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns, index=X.index)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

print("\n✅ Feature scaling completed for linear models")
print(f"Scaled feature means: {X_scaled.mean().round(3).tolist()}")
print(f"Scaled feature stds: {X_scaled.std().round(3).tolist()}")

In [ ]:
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=None
)

# Also split scaled features
X_train_scaled, X_val_scaled, _, _ = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=None
)

print("DATA SPLITTING SUMMARY")
print("="*22)
print(f"Training set: {X_train.shape[0]} samples")
print(f"Validation set: {X_val.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"Features: {X_train.shape[1]}")

# Verify no data leakage
print(f"\nTarget statistics:")
print(f"Training mean: {y_train.mean():.2f} ± {y_train.std():.2f}")
print(f"Validation mean: {y_val.mean():.2f} ± {y_val.std():.2f}")
print(f"Full dataset mean: {y.mean():.2f} ± {y.std():.2f}")

## 5. Model Training and Evaluation

In [ ]:
# Define evaluation metrics function
def evaluate_model(y_true, y_pred, model_name):
    """Calculate and return evaluation metrics"""
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred) * 100
    
    metrics = {
        'Model': model_name,
        'MAE': mae,
        'RMSE': rmse,
        'R²': r2,
        'MAPE (%)': mape
    }
    
    return metrics

# Initialize results storage
model_results = []
model_predictions = {}
trained_models = {}

print("🚀 STARTING MODEL TRAINING")
print("="*30)

In [ ]:
# Model 1: Random Forest Regressor
print("\n🌲 Training Random Forest Regressor...")
print("-" * 40)

# Train Random Forest
rf_model = RandomForestRegressor(
    n_estimators=200,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_val)

# Evaluate Random Forest
rf_metrics = evaluate_model(y_val, y_pred_rf, 'Random Forest')
model_results.append(rf_metrics)
model_predictions['Random Forest'] = y_pred_rf
trained_models['Random Forest'] = rf_model

print(f"Random Forest Results:")
for metric, value in rf_metrics.items():
    if metric != 'Model':
        print(f"  {metric}: {value:.4f}")

# Feature importance for Random Forest
rf_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

print(f"\nTop 5 Most Important Features:")
for i, (_, row) in enumerate(rf_importance.head().iterrows()):
    print(f"  {i+1}. {row['Feature']}: {row['Importance']:.4f}")

In [ ]:
# Model 2: XGBoost Regressor
print("\n🚀 Training XGBoost Regressor...")
print("-" * 35)

# Train XGBoost
xgb_model = xgb.XGBRegressor(
    n_estimators=200,
    max_depth=8,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    verbosity=0
)

xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_val)

# Evaluate XGBoost
xgb_metrics = evaluate_model(y_val, y_pred_xgb, 'XGBoost')
model_results.append(xgb_metrics)
model_predictions['XGBoost'] = y_pred_xgb
trained_models['XGBoost'] = xgb_model

print(f"XGBoost Results:")
for metric, value in xgb_metrics.items():
    if metric != 'Model':
        print(f"  {metric}: {value:.4f}")

# Feature importance for XGBoost
xgb_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': xgb_model.feature_importances_
}).sort_values('Importance', ascending=False)

print(f"\nTop 5 Most Important Features:")
for i, (_, row) in enumerate(xgb_importance.head().iterrows()):
    print(f"  {i+1}. {row['Feature']}: {row['Importance']:.4f}")

In [ ]:
# Model 3: Ridge Regression
print("\n📏 Training Ridge Regression...")
print("-" * 32)

# Train Ridge Regression (using scaled features)
ridge_model = Ridge(alpha=1.0, random_state=42)
ridge_model.fit(X_train_scaled, y_train)
y_pred_ridge = ridge_model.predict(X_val_scaled)

# Evaluate Ridge Regression
ridge_metrics = evaluate_model(y_val, y_pred_ridge, 'Ridge Regression')
model_results.append(ridge_metrics)
model_predictions['Ridge Regression'] = y_pred_ridge
trained_models['Ridge Regression'] = ridge_model

print(f"Ridge Regression Results:")
for metric, value in ridge_metrics.items():
    if metric != 'Model':
        print(f"  {metric}: {value:.4f}")

# Feature coefficients for Ridge Regression
ridge_coeffs = pd.DataFrame({
    'Feature': X_train_scaled.columns,
    'Coefficient': ridge_model.coef_
}).sort_values('Coefficient', key=abs, ascending=False)

print(f"\nTop 5 Most Important Features (by coefficient magnitude):")
for i, (_, row) in enumerate(ridge_coeffs.head().iterrows()):
    print(f"  {i+1}. {row['Feature']}: {row['Coefficient']:.4f}")

## 6. Model Comparison and Selection

In [ ]:
# Comprehensive model comparison
print("📊 MODEL COMPARISON SUMMARY")
print("="*30)

# Create results DataFrame
results_df = pd.DataFrame(model_results)
results_df = results_df.set_index('Model')

# Display results table
print("\nModel Performance Comparison:")
display(results_df.round(4))

# Find best model for each metric
print("\nBest Models by Metric:")
print(f"  Lowest MAE: {results_df['MAE'].idxmin()} ({results_df['MAE'].min():.4f})")
print(f"  Lowest RMSE: {results_df['RMSE'].idxmin()} ({results_df['RMSE'].min():.4f})")
print(f"  Highest R²: {results_df['R²'].idxmax()} ({results_df['R²'].max():.4f})")
print(f"  Lowest MAPE: {results_df['MAPE (%)'].idxmin()} ({results_df['MAPE (%)'].min():.4f}%)")

# Overall best model (based on R²)
best_model_name = results_df['R²'].idxmax()
best_model = trained_models[best_model_name]
best_predictions = model_predictions[best_model_name]

print(f"\n🏆 Overall Best Model: {best_model_name}")
print(f"   R² Score: {results_df.loc[best_model_name, 'R²']:.4f}")

In [ ]:
# Visualize model performance comparison
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Performance metrics comparison
metrics_to_plot = ['MAE', 'RMSE', 'R²', 'MAPE (%)']
colors = ['skyblue', 'lightcoral', 'lightgreen']

for i, metric in enumerate(metrics_to_plot):
    ax = axes[i//3, i%3] if i < 3 else axes[1, i-3]
    bars = ax.bar(results_df.index, results_df[metric], color=colors, alpha=0.7, edgecolor='black')
    ax.set_title(f'{metric} Comparison', fontweight='bold')
    ax.set_ylabel(metric)
    ax.tick_params(axis='x', rotation=45)
    
    # Add value labels on bars
    for bar, value in zip(bars, results_df[metric]):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
    
    ax.grid(True, alpha=0.3)

# Prediction vs Actual scatter plots
for i, (model_name, predictions) in enumerate(model_predictions.items()):
    ax = axes[1, i] if i < 3 else None
    if ax is not None:
        ax.scatter(y_val, predictions, alpha=0.6, color=colors[i])
        ax.plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--', lw=2)
        ax.set_xlabel('Actual Calories')
        ax.set_ylabel('Predicted Calories')
        ax.set_title(f'{model_name}\nPredictions vs Actual')
        ax.grid(True, alpha=0.3)
        
        # Calculate and display R²
        r2 = r2_score(y_val, predictions)
        ax.text(0.05, 0.95, f'R² = {r2:.3f}', transform=ax.transAxes, 
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8),
                verticalalignment='top', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Feature importance comparison for tree-based models
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# Random Forest feature importance
axes[0].barh(range(len(rf_importance.head(10))), rf_importance.head(10)['Importance'], 
             color='skyblue', alpha=0.7)
axes[0].set_yticks(range(len(rf_importance.head(10))))
axes[0].set_yticklabels(rf_importance.head(10)['Feature'])
axes[0].set_xlabel('Feature Importance')
axes[0].set_title('Random Forest\nTop 10 Feature Importances', fontweight='bold')
axes[0].grid(True, alpha=0.3)

# XGBoost feature importance
axes[1].barh(range(len(xgb_importance.head(10))), xgb_importance.head(10)['Importance'], 
             color='lightcoral', alpha=0.7)
axes[1].set_yticks(range(len(xgb_importance.head(10))))
axes[1].set_yticklabels(xgb_importance.head(10)['Feature'])
axes[1].set_xlabel('Feature Importance')
axes[1].set_title('XGBoost\nTop 10 Feature Importances', fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Ridge regression coefficients
plt.figure(figsize=(12, 8))
plt.barh(range(len(ridge_coeffs.head(10))), ridge_coeffs.head(10)['Coefficient'], 
         color='lightgreen', alpha=0.7)
plt.yticks(range(len(ridge_coeffs.head(10))), ridge_coeffs.head(10)['Feature'])
plt.xlabel('Coefficient Value')
plt.title('Ridge Regression\nTop 10 Feature Coefficients (by magnitude)', fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Cross-validation for robust model evaluation
print("🔄 CROSS-VALIDATION ANALYSIS")
print("="*30)

# Perform 5-fold cross-validation for all models
cv_results = {}

for model_name, model in trained_models.items():
    print(f"\nPerforming CV for {model_name}...")
    
    if model_name == 'Ridge Regression':
        # Use scaled features for Ridge
        cv_scores = cross_val_score(model, X_scaled, y, cv=5, scoring='r2', n_jobs=-1)
    else:
        # Use original features for tree-based models
        cv_scores = cross_val_score(model, X, y, cv=5, scoring='r2', n_jobs=-1)
    
    cv_results[model_name] = cv_scores
    
    print(f"  CV R² Scores: {[f'{score:.4f}' for score in cv_scores]}")
    print(f"  Mean CV R²: {cv_scores.mean():.4f} (±{cv_scores.std()*2:.4f})")

# Create cross-validation comparison plot
plt.figure(figsize=(12, 6))
box_data = [cv_results[model] for model in cv_results.keys()]
box_plot = plt.boxplot(box_data, labels=list(cv_results.keys()), patch_artist=True)

# Color the boxes
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

plt.title('Cross-Validation R² Scores Comparison', fontweight='bold', fontsize=14)
plt.ylabel('R² Score')
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Statistical summary of CV results
print("\nCross-Validation Summary:")
cv_summary = pd.DataFrame({
    'Mean_R2': [cv_results[model].mean() for model in cv_results.keys()],
    'Std_R2': [cv_results[model].std() for model in cv_results.keys()],
    'Min_R2': [cv_results[model].min() for model in cv_results.keys()],
    'Max_R2': [cv_results[model].max() for model in cv_results.keys()]
}, index=list(cv_results.keys()))

display(cv_summary.round(4))

In [ ]:
# Generate predictions on test set using the best model
print(f"🎯 GENERATING FINAL PREDICTIONS")
print("="*33)

print(f"Using best model: {best_model_name}")
print(f"Best model R² on validation: {results_df.loc[best_model_name, 'R²']:.4f}")

# Generate test predictions
if best_model_name == 'Ridge Regression':
    test_predictions = best_model.predict(X_test_scaled)
else:
    test_predictions = best_model.predict(X_test)

print(f"\nTest predictions generated: {len(test_predictions)} samples")
print(f"Prediction range: {test_predictions.min():.2f} - {test_predictions.max():.2f}")
print(f"Prediction mean: {test_predictions.mean():.2f}")
print(f"Prediction std: {test_predictions.std():.2f}")

# Create submission file
submission = pd.DataFrame({
    'id': test_data['id'],
    'Calories': test_predictions
})

# Save submission
submission.to_csv('calorie_predictions.csv', index=False)
print(f"\n✅ Predictions saved to 'calorie_predictions.csv'")
print(f"Submission shape: {submission.shape}")
display(submission.head(10))

## 7. Summary and Conclusions

### Key Findings:

1. **Best Performing Model**: The analysis will identify which of the three models (Random Forest, XGBoost, or Ridge Regression) performs best on this dataset.

2. **Feature Importance**: Tree-based models provide insights into which features are most predictive of calorie expenditure.

3. **Model Characteristics**:
   - **Random Forest**: Robust ensemble method, good for mixed data types
   - **XGBoost**: Advanced gradient boosting, often superior performance
   - **Ridge Regression**: Simple linear model, good baseline and interpretability

4. **Cross-Validation**: Provides robust estimates of model performance and helps identify overfitting.

### Next Steps:
- Consider hyperparameter tuning for the best model
- Explore additional feature engineering
- Try ensemble methods combining multiple models
- Analyze prediction errors for insights into model improvements